In [4]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
# from sklearn.cross_validation import train_test_split, KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob

#from keras import Sequential
#from keras.layers import Embedding, LSTM, Dense, Dropout
%matplotlib inline

In [6]:
df = pd.read_csv('/Users/menusha/Desktop/thamashi_data_1.csv', encoding="latin-1")
df['label'] = df['sentiment'].map({'usability': 0,'stability': 1})
X = df['review']
y = df['label']

seed = 7

def bigram_word_feats(wordsarray):
    #Converting the array of posts to a string
    words="".join(wordsarray)
    words.lower()
    #Splitting the post to bigrams
    finder1 = BigramCollocationFinder.from_words((words.lower()).split())
    #Filtering by bigrams which have been repeated at least thrice
    finder1.apply_freq_filter(3)
    bigram_measures = BigramAssocMeasures()
    #returning bigrams according to scores of chi square
    return([" ".join(d[0]) for d in finder1.score_ngrams(bigram_measures.chi_sq)])
# TfidfVectorizer is the powerful equivalent of CountVectorizer for TF-IDF weights
def tfidf_extractor(corpus, ngram_range=(1,1)):
    #
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

lbig = {}
#variable for POS
raw =[]

lbig = bigram_word_feats(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 21)



nb = Pipeline([("features", FeatureUnion([('tfidf', TfidfVectorizer(min_df=1, binary = True, use_idf = False, norm = False)),('bigram',  CountVectorizer(ngram_range =(2, 2), vocabulary = lbig,  binary = True))
                                                                             ])),('clf', MultinomialNB()),('text',TextBlob())])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)
my_tags = ['usability','stability']
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))



TypeError: __init__() missing 1 required positional argument: 'text'